<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220222_sm_TPU_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full training

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 78.3 MB/s 
     |████████████████████████████████| 133 kB 89.8 MB/s 
     |████████████████████████████████| 243 kB 90.8 MB/s 
     |████████████████████████████████| 1.1 MB 79.6 MB/s 
     |████████████████████████████████| 67 kB 6.1 MB/s 
     |████████████████████████████████| 144 kB 74.4 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 91.2 MB/s 
     |████████████████████████████████| 596 kB 79.0 MB/s 
     |████████████████████████████████| 6.8 MB 89.9 MB/s 
     |████████████████████████████████| 895 kB 85.3 MB/s 
     |████████████████████████████████| 1.2 MB 77.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 3.1 MB/s 
     |████████████████████████████████| 149.9 

In [3]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from accelerate import Accelerator

In [4]:
import os
import random
from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc


In [5]:
# accelerator = Accelerator()
# device = accelerator.device

In [6]:
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli[:100000]

In [7]:
PATH =  '/content/drive/MyDrive/KLUE'
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_train = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_train.json')
temp = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')

klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_train = klue_train[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)
klue_train.rename(columns = {'gold_label':'label'}, inplace=True)

train = pd.concat([kakao_dev,kakao_test, kakao_snli])
train.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)
train = pd.concat([temp, train, klue_dev, klue_train], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
159516,159516,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",개발 음식 활용 혜택만 주어진다.,contradiction
159517,159517,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",힐링푸드 전문가에게 혜택이 주어진다.,neutral
159518,159518,"힐링푸드 전문가 양성 교육 참여, 개발 음식 활용 등의 혜택도 주어진다.",혜택에는 힐링푸드 전문가 양성 교육 참여가 있다.,entailment
159519,159519,힛걸 진심 최고다 그 어떤 히어로보다 멋지다,힛걸 그 어떤 히어로보다 별로다.,contradiction


In [8]:
from transformers import AutoTokenizer

checkpoint = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/734k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [9]:
train_dataset, eval_dataset = train_test_split(train, test_size=0.2, shuffle=False)

tokenized_train = tokenizer(
    list(train_dataset['premise']),
    list(train_dataset['hypothesis']),
    return_tensors="pt",
    max_length=128, # Max_Length = 190
    padding=True,
    truncation=True,
    add_special_tokens=True
)

tokenized_eval = tokenizer(
    list(eval_dataset['premise']),
    list(eval_dataset['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

In [10]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [11]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label


train_label = label_to_num(train_dataset['label'].values)
eval_label = label_to_num(eval_dataset['label'].values)

In [12]:
train_dataset = BERTDataset(tokenized_train, train_label)
eval_dataset = BERTDataset(tokenized_eval, eval_label)

In [13]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    0, 14441,  2073, 12382, 13169,  2200,  3797, 21505,  9005,  2259,
          3997,  2031,  2079,  3661, 31221,  5845,  2200,  2112,    16,  5950,
         15351, 17788,  7285,   748,  2088, 22048,  2470,  1132, 21893, 15351,
          6481, 27135,  5417,  4084,  1972,  2145, 17524,  2138, 15526,  2259,
           575, 28674,    18,     2, 14441,  2079,  3883,  2031,  2079,  5845,
         28674,    18,     2,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     

In [14]:
# tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
# tokenized_datasets.set_format("torch")
# tokenized_datasets["train"].column_names

In [15]:
# train_dataset = train_dataset.rename_column("label", "labels")

In [16]:
# ["attention_mask", "input_ids", "labels", "token_type_ids"]

In [17]:
from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     train_dataset, shuffle=True, batch_size=16
# )
# eval_dataloader = DataLoader(
#     eval_dataset, batch_size=16
# )

In [18]:
# for batch in train_dataloader:
#     break
# {k: v.shape for k, v in batch.items()}

In [19]:
from transformers import AutoModel, AutoModelForSequenceClassification

# config = AutoConfig.from_pretrained(checkpoint)
# config.num_labels = 3

# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)
# model = AutoModel.from_pretrained("klue/roberta-large", num_labels=3)

In [20]:
# outputs = model(**batch)
# print(outputs.loss, outputs.logits.shape)

In [21]:
from transformers import AdamW

# optimizer = AdamW(model.parameters(), lr=	0.0001)

In [22]:
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup

# num_epochs = 3
# num_training_steps = num_epochs * len(train_dataloader)
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )
# print(num_training_steps)

In [23]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# device

In [24]:
# model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

In [25]:
# import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# device

In [26]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
log_interval = 100

In [28]:
from tqdm.auto import tqdm
from datasets import load_metric
# progress_bar = tqdm(range(num_training_steps))
# metric = load_metric("accuracy")

def training_function():

  import warnings
  warnings.filterwarnings('ignore')

  accelerator = Accelerator()

  train_dataloader = DataLoader(
    train_dataset, shuffle=True, batch_size=16
  )
  eval_dataloader = DataLoader(
      eval_dataset, batch_size=16
  )

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 6
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          # batch = {k: v.to(device) for k, v in batch.items()}
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          # print(batch)
          # loss.backward()
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          # print(outputs)
          # print('-----------')
          # print(batch['labels'])
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
          # if batch_id  == 100:
          #     print("epoch {} batch id {} loss {} train acc {}".format(epoch+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        # batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        # logits = outputs.logits
        # predictions = torch.argmax(logits, dim=-1)
        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
        # metric.add_batch(predictions=predictions, references=batch["labels"])
      # print('epoch : ' + str(epoch))
      # accuracy = metric.compute()
      # print(accuracy)
  gc.collect()
  model.save_pretrained('./result/best_model')

In [29]:
gc.collect()

50

In [ ]:
from accelerate import notebook_launcher
import warnings

warnings.filterwarnings('ignore')
notebook_launcher(training_function)

Launching a training on 8 TPU cores.


In [ ]:
print(my_variable)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Tokenizer_NAME = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(Tokenizer_NAME)

MODEL_NAME = './result/best_model'
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
model.resize_token_embeddings(tokenizer.vocab_size)
model.to(device)

print(tokenizer)

In [ ]:
test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

print(test_dataset.__len__())
print(test_dataset.__getitem__(1665))
print(tokenizer.decode(test_dataset.__getitem__(6)['input_ids']))

In [ ]:
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

model.eval()
output_pred = []
output_prob = []

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'].to(device),
            attention_mask=data['attention_mask'].to(device),
            token_type_ids=data['token_type_ids'].to(device)
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

In [ ]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(pred_answer)
print(answer)

In [ ]:
df = pd.DataFrame(answer, columns=['index', 'label'])

df.to_csv('/content/drive/MyDrive/KLUE/submission.csv', index=False)

print(df)